In [1]:
from pyspark import SparkConf, SparkContext
#스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_02_partition")
spark = SparkContext(conf=conf).getOrCreate()
spark

24/12/06 18:05:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local appName=241205_02_partition>

# 사용자가 지정하는 파티션
```
partitionBy()


In [2]:
pairs = spark.parallelize(
    [
        1,2,3,4,2,4,1
    ]
).map(lambda x: (x,x))

In [3]:
pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]

In [4]:
pairs.glom().collect()

[[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]]

In [5]:
#파티션 현황 확인 glom()

pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [6]:
# 파티션별로 규칙에 따라 데이터를 넣을 수 있다. > Lambda 식으로 정의
#파티션 배치 함수

pairs.partitionBy(3, lambda x:x%2).glom().collect() #짝수/홀수를 기준으로 나눔 

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)], []]

In [7]:
new_pairs = pairs.partitionBy(2, lambda x:x%2).persist()
new_pairs.glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [8]:
spark.stop()